# 03 - Model Training with Model Registry

## Train and Deploy an XGBoost Compliance Classifier

**Goal:** Train a binary compliance classifier and register it in Snowflake's Model Registry.

### What is the Model Registry?

The Model Registry provides:
- **Version Control:** Track model versions with metadata
- **Deployment:** Deploy models for SQL-based inference  
- **Metrics Tracking:** Store accuracy, precision, recall alongside models
- **Audit Trail:** Know which model made which prediction

### Why XGBoost?

| Property | Benefit for Compliance |
|----------|----------------------|
| **Fast inference** | Classify millions of emails quickly |
| **Interpretable** | Feature importances explain decisions |
| **Handles imbalance** | Works well with minority class violations |
| **Native to Snowpark ML** | Runs entirely inside Snowflake |

---

In [ ]:
# Setup
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.xgboost import XGBClassifier

session = Session.builder.getOrCreate()
session.use_warehouse("COMPLIANCE_DEMO_WH")
session.use_database("COMPLIANCE_DEMO")
session.use_schema("ML")

print(f"Connected as: {session.get_current_user()}")

## 1. Prepare Training Data

Load features from the Feature Store and create binary labels (VIOLATION vs CLEAN).

In [ ]:
# Load features from Feature Store
training_df = session.table("COMPLIANCE_DEMO.ML.EMAIL_RISK_FEATURES")

print("--- Training Data Preview ---")
training_df.show(5)
print(f"Total records: {training_df.count():,}")

# Check label distribution
print("\n--- Label Distribution ---")
training_df.group_by("COMPLIANCE_LABEL").count().show()

In [ ]:
# Create binary labels: anything that's not CLEAN is a violation
training_df = training_df.with_column(
    "IS_VIOLATION",
    (col("COMPLIANCE_LABEL") != "CLEAN").cast("INT")
)

print("--- Binary Label Distribution ---")
training_df.group_by("IS_VIOLATION").count().show()

# Define feature columns and label
feature_cols = [
    "URGENCY_SCORE",
    "SECRECY_SCORE", 
    "TOTAL_RISK_SCORE",
    "BODY_LENGTH",
    "IS_CROSS_DEPT",
    "IS_BARRIER_CROSSING",
    "IS_AFTER_HOURS",
]
label_col = "IS_VIOLATION"

## 2. Train/Test Split and Model Training

Split data and train an XGBoost classifier. All computation happens inside Snowflake.

In [ ]:
# Select features and split data
model_df = training_df.select("EMAIL_ID", *feature_cols, label_col, "COMPLIANCE_LABEL")
train_df, test_df = model_df.random_split([0.8, 0.2], seed=42)

print(f"Training set: {train_df.count():,} records")
print(f"Test set: {test_df.count():,} records")

# Train XGBoost classifier
print("\n🏋️ Training XGBoost classifier...")

xgb_model = XGBClassifier(
    input_cols=feature_cols,
    label_cols=[label_col],
    output_cols=["PREDICTED_VIOLATION"],
    n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
)

xgb_model.fit(train_df)
print("✅ Model training complete")

## 3. Evaluate Model Performance

Calculate precision, recall, and confusion matrix on the test set.

In [ ]:
# Generate predictions on test set
predictions_df = xgb_model.predict(test_df)

# Calculate metrics
predictions_pd = predictions_df.select("IS_VIOLATION", "PREDICTED_VIOLATION").to_pandas()
y_true = predictions_pd["IS_VIOLATION"]
y_pred = predictions_pd["PREDICTED_VIOLATION"]

# Calculate confusion matrix components
tp = ((y_true == 1) & (y_pred == 1)).sum()
fp = ((y_true == 0) & (y_pred == 1)).sum()
fn = ((y_true == 1) & (y_pred == 0)).sum()
tn = ((y_true == 0) & (y_pred == 0)).sum()

accuracy = (y_true == y_pred).mean()
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("=" * 50)
print("MODEL PERFORMANCE METRICS")
print("=" * 50)
print(f"  Accuracy:  {accuracy:.2%}")
print(f"  Precision: {precision:.2%} (of flagged, how many are violations)")
print(f"  Recall:    {recall:.2%} (of violations, how many did we catch)")
print(f"  F1 Score:  {f1:.2%}")

print("\n--- Confusion Matrix ---")
print(f"                    Predicted")
print(f"                 CLEAN   VIOLATION")
print(f"Actual CLEAN     {tn:5d}   {fp:5d}")
print(f"Actual VIOLATION {fn:5d}   {tp:5d}")

## 4. Register Model in Model Registry

Save the trained model to Snowflake's Model Registry for versioning and deployment.

In [ ]:
# Initialize Model Registry
reg = Registry(session=session, database_name="COMPLIANCE_DEMO", schema_name="ML")

# Register the model with metrics
model_name = "EMAIL_COMPLIANCE_CLASSIFIER"

mv = reg.log_model(
    model=xgb_model,
    model_name=model_name,
    version_name="V1",
    metrics={
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1_score": float(f1),
        "training_samples": train_df.count(),
        "test_samples": test_df.count(),
    },
    comment="XGBoost classifier for email compliance violation detection. Binary classification: CLEAN vs VIOLATION."
)

print(f"✅ Registered model: {model_name}/V1")

# Show registered models
print("\n--- Registered Models ---")
reg.show_models()

## 5. Run Inference with Registered Model

Load the model from the registry and score all emails.

In [ ]:
# Load model from registry and run inference
loaded_model = reg.get_model(model_name).version("V1")

# Score all emails
all_emails_df = session.table("COMPLIANCE_DEMO.ML.EMAIL_RISK_FEATURES").select(
    "EMAIL_ID",
    "COMPLIANCE_LABEL",
    *feature_cols,
)

scored_df = loaded_model.run(all_emails_df, function_name="predict")

print("--- Scored Emails Preview ---")
scored_df.select("EMAIL_ID", "COMPLIANCE_LABEL", "PREDICTED_VIOLATION").show(10)

# Save predictions
scored_df.write.mode("overwrite").save_as_table("COMPLIANCE_DEMO.ML.MODEL_PREDICTIONS_V1")
print("✅ Saved predictions to COMPLIANCE_DEMO.ML.MODEL_PREDICTIONS_V1")

## Summary

**What we accomplished:**
- Trained an XGBoost classifier on 8,000 emails (80% of dataset)
- Evaluated performance: precision, recall, F1, confusion matrix
- Registered the model in Snowflake's Model Registry with metrics
- Ran inference on all 10,000 emails and saved predictions

**Model Registered:**
- `EMAIL_COMPLIANCE_CLASSIFIER/V1`
- Binary classification: CLEAN (0) vs VIOLATION (1)
- Features: urgency, secrecy, risk scores, cross-dept, barrier-crossing, after-hours

**Next:** In notebook 04, we'll integrate this ML model with Claude claude-opus-4-5 to build an optimal tiered architecture.